# To-Dos

- establish system to name batches provided by sascha
- establish loop for changing the bboxes on all coco json if they havent been changed so far




In [1]:
import cv2, os, math, sys
from pathlib import Path
from typing import List, Tuple
import numpy as np
from tqdm import tqdm
from pathlib import Path
from typing import List, Dict, Any
import os, json, shutil, random, math, datetime as dt
import pandas as pd


VERBOSE = True

def debug(msg: str, level: str = 'INFO') -> None:
    if not VERBOSE:
        return
    colours = {'INFO': '\033[94m', 'WARN': '\033[93m', 'ERR': '\033[91m'}
    print(f"{colours.get(level, '')}[{level}] {msg}\033[0m")

## 1 · Configuration

In [2]:
SRC_IMG_DIR         = Path('../data/rotation/batches/batch_20250703_01/images/default')
SRC_ANNOTATION_PATH = Path('../data/rotation/batches/batch_20250703_01/annotations/instances_default.json')

DEST_IMG_DIR        = Path('../data/rotation/batches/batch_20250703_01/images/boxes')
DEST_IMG_DIR.mkdir(parents=True, exist_ok=True)

ANGLES: List[int]   = [0, 45, 90, 180, 270, 360]

debug('Config loaded.')

[INFO] Config loaded.


## 2 · COCO helpers & utils

In [ ]:

def load_coco(json_path: Path) -> Dict[str, Any]:
    with open(json_path, 'r', encoding='utf-8') as f:
        coco = json.load(f)
    return coco

    

def create_obb_tuple(anns):
    bbox = anns.get("bbox", "No bbox found")
    if(len(bbox) == 4):    
        x, y, w, h = anns["bbox"]
        cx = x + (w/2)
        cy = y + (h/2)
        angle = anns["attributes"].get("rotation", 0.0)
        obb_list = [cx, cy, w, h, angle]
        anns["bbox"] = obb_list
    else: 
        #print(f"type = {type(bbox)}, length = {len(bbox) if hasattr(bbox, '__len__') else 'N/A'}, bbox: {bbox}")
        print("Weirdle after every element is on 5 tuples it starts to iterate again")



def replace_obb(coco, batchnames):
    for anns in coco['annotations']:
        create_obb_tuple(anns)
        
    json_object = json.dumps(coco)
    with open(f"../data/rotation/batches/{batchnames}/annotations/instances_updated.json", mode="w") as file:
        file.write(json_object)
              
        
        
    

## Debug

In [4]:
def lookAtCoco(json_path: Path):
    coco = load_coco(json_path)
    print(f"Coco {coco}")
    print(f"Keys {coco.keys()}")
    print(f"Values {coco.values()}")
    print(f"Items {coco.items()}")

    print(f"Images      : {(coco['images'])}")
    print(f"Annotations : {(coco['annotations'])}")
    print(f"Categories  : {(coco['categories'])}\n")
    
    
    
    
def debug_bbox(coco):
    for i, anns in enumerate(coco['annotations']):
        bbox = anns.get("bbox", "No bbox found")
        print(f"Annotation {i}: bbox = {bbox}, type = {type(bbox)}, length = {len(bbox) if hasattr(bbox, '__len__') else 'N/A'}")

In [5]:
coco = load_coco(json_path=SRC_ANNOTATION_PATH)

In [7]:
lookAtCoco(json_path=SRC_ANNOTATION_PATH)

Coco {'licenses': [{'name': '', 'id': 0, 'url': ''}], 'info': {'contributor': '', 'date_created': '', 'description': '', 'url': '', 'version': '', 'year': ''}, 'categories': [{'id': 1, 'name': 'isOk', 'supercategory': ''}, {'id': 2, 'name': 'inner-edge-', 'supercategory': ''}, {'id': 3, 'name': 'additional-infoblock', 'supercategory': ''}, {'id': 4, 'name': 'inner-edge-tolerance', 'supercategory': ''}, {'id': 5, 'name': 'infoblock', 'supercategory': ''}, {'id': 6, 'name': 'gdt', 'supercategory': ''}, {'id': 7, 'name': 'text', 'supercategory': ''}, {'id': 8, 'name': 'additional infoblock', 'supercategory': ''}, {'id': 9, 'name': 'surface-roughness', 'supercategory': ''}, {'id': 10, 'name': 'welding_symbol', 'supercategory': ''}, {'id': 11, 'name': 'infoBlock', 'supercategory': ''}, {'id': 12, 'name': 'general-chamfer', 'supercategory': ''}, {'id': 13, 'name': 'drawing-area', 'supercategory': ''}], 'images': [{'id': 1, 'width': 3055, 'height': 2160, 'file_name': '10000.png', 'license': 0

In [8]:
new_obb = replace_obb(coco)

## 3 · Geometry with loss stats

In [20]:
def rectify_patch(img, cx, cy, w, h, theta):
    print(f'Rectify: centre=({cx:.1f},{cy:.1f}) w={w:.1f} h={h:.1f} θ={theta:.1f}')
    
    M = cv2.getRotationMatrix2D((cx, cy), theta, 1.0)
    
    print(M)
    
    print(f"img.shape[1]: {img.shape[1]}")
    print(f"img.shape[0]: {img.shape[0]}")

    
    rot = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    
    cv2.imwrite('rotated_image.jpg', rot)
    
image_file = '../data/rotation/batches/batch_20250703_01/images/default/10000.png'
image_1 = cv2.imread(image_file)


rectify_patch(image_1, 2452.195, 549.365, 35.37, 32.47, 314.6)


Rectify: centre=(2452.2,549.4) w=35.4 h=32.5 θ=314.6
[[ 7.02153053e-01 -7.12026046e-01  1.12154100e+03]
 [ 7.12026046e-01  7.02153053e-01 -1.58240007e+03]]
img.shape[1]: 3055
img.shape[0]: 2160


In [25]:
print(type(image_1))        # <class 'numpy.ndarray'>
print(image_1.dtype)  

<class 'numpy.ndarray'>
uint8


In [26]:
def crop_oriented_bbox(img, cx, cy, w, h, theta):
    # Step 1: Rotate the entire image around the bbox center
    M = cv2.getRotationMatrix2D((cx, cy), theta, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    
    # Step 2: Crop the now-aligned rectangle
    x1 = int(cx - w/2)
    y1 = int(cy - h/2)
    x2 = int(cx + w/2)
    y2 = int(cy + h/2)
    
    # Ensure bounds are within image
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(img.shape[1], x2), min(img.shape[0], y2)
    
    cropped = rotated[y1:y2, x1:x2]
    return cropped

In [27]:
cv2.imwrite('cropped_image.jpg', crop_oriented_bbox(image_1, 2452.195, 549.365, 35.37, 32.47, 314.6))


True

In [3]:
import cv2
import os


if not os.path.exists(image_file):
    print(f"Error: The image file {image_file} does not exist.")
else:
    print("Image file exists. Attempting to read the image...")
    image_1 = cv2.imread(image_file)
    if image_1 is None:
        print("Error: Could not read the image.")
    else:
        print("Image read successfully. Attempting to display the image...")
        cv2.imshow("test", image_1)
        print("Image displayed. Press any key to close the window...")

        while True:
            key = cv2.waitKey(1) & 0xFF
            if key != 255:
                print(f"Key pressed: {key}. Closing the window...")
                break

        cv2.destroyAllWindows()
        print("Window closed.")


Image file exists. Attempting to read the image...
Image read successfully. Attempting to display the image...
Image displayed. Press any key to close the window...


2025-07-08 14:07:58.162 python[69462:8755635] +[IMKClient subclass]: chose IMKClient_Legacy
2025-07-08 14:07:58.162 python[69462:8755635] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Key pressed: 102. Closing the window...
Window closed.


Rectify: centre=(2452.2,549.4) w=35.4 h=32.5 θ=314.6
[[ 7.02153053e-01  7.12026046e-01  3.39216633e+02]
 [-7.12026046e-01  7.02153053e-01  1.90965344e+03]]


In [13]:
def rotate_patch(patch, angle):
    h, w = patch.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    cos, sin = abs(M[0,0]), abs(M[0,1])
    new_w, new_h = int(h*sin + w*cos), int(h*cos + w*sin)
    M[0,2] += new_w/2 - w/2
    M[1,2] += new_h/2 - h/2
    return cv2.warpAffine(patch, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderValue=255)

## 4 · Save helper

## 5 · Overlay helper (optional)

## 6 · Main loop with extra prints

In [14]:
def process_dataset(src_img_dir: Path, coco_json: Path, dest_img_dir: Path,
                    angles: List[int] = ANGLES, every: int = 200):
    coco = load_coco(coco_json)
    


## 7 · Interactive inspect()

## 8 · Smoke test

## 9 · Run full dataset

In [15]:
# VERBOSE = False  # uncomment for speed
# process_dataset(SRC_IMG_DIR, SRC_ANNOTATION_PATH, DEST_IMG_DIR)

intersting features to add>


Extra diagnostics to explain **where cropping goes wrong** and to display key COCO fields for every patch:
* verbose per‑annotation banner with image/annotation IDs & bbox details
* lost‑pixels report when a crop gets clipped by image borders
* `inspect()` prints the category, bbox, rotation and lost‑pixel stats
* helper `overlay_bbox()` overlays the rotated‑back bbox for visual sanity checks